In [1]:
print("hello")

hello


In [3]:
from langchain_community.document_loaders import GithubFileLoader

In [4]:
import subprocess
# Run the bash script and capture its environment
result = subprocess.run(
    ["bash", "-c", "source set_token.sh && env"],
    stdout=subprocess.PIPE,
    text=True
)

In [6]:
# Parse the environment variables from the output
env_vars = {}
for line in result.stdout.splitlines():
    if "=" in line:
        key, value = line.split("=", 1)
        env_vars[key] = value

# retrieve the token
access_token = env_vars.get("GITHUB_ACCESS_TOKEN")

In [7]:
loader = GithubFileLoader(
    repo="nicbarker/clay",  # the repo name
    branch="main",  # the branch name
    access_token=access_token,
    github_api_url="https://api.github.com",
    file_filter=lambda file_path: file_path.endswith(
        ".md"
    ),  # load all markdowns files.
)
documents = loader.load()

In [8]:
documents

[Document(metadata={'path': 'LICENSE.md', 'sha': '2968c61bcdc6ecbef470e02edd45b47aba1d8c44', 'source': 'https://api.github.com/nicbarker/clay/blob/main/LICENSE.md'}, page_content="zlib/libpng license\n\nCopyright (c) 2024 Nic Barker\n\nThis software is provided 'as-is', without any express or implied warranty.\nIn no event will the authors be held liable for any damages arising from the\nuse of this software.\n\nPermission is granted to anyone to use this software for any purpose,\nincluding commercial applications, and to alter it and redistribute it\nfreely, subject to the following restrictions:\n\n    1. The origin of this software must not be misrepresented; you must not\n    claim that you wrote the original software. If you use this software in a\n    product, an acknowledgment in the product documentation would be\n    appreciated but is not required.\n\n    2. Altered source versions must be plainly marked as such, and must not\n    be misrepresented as being the original soft

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=30)

chunked_docs = splitter.split_documents(documents)

In [10]:
chunked_docs[:5]

[Document(metadata={'path': 'LICENSE.md', 'sha': '2968c61bcdc6ecbef470e02edd45b47aba1d8c44', 'source': 'https://api.github.com/nicbarker/clay/blob/main/LICENSE.md'}, page_content="zlib/libpng license\n\nCopyright (c) 2024 Nic Barker\n\nThis software is provided 'as-is', without any express or implied warranty.\nIn no event will the authors be held liable for any damages arising from the\nuse of this software.\n\nPermission is granted to anyone to use this software for any purpose,\nincluding commercial applications, and to alter it and redistribute it\nfreely, subject to the following restrictions:"),
 Document(metadata={'path': 'LICENSE.md', 'sha': '2968c61bcdc6ecbef470e02edd45b47aba1d8c44', 'source': 'https://api.github.com/nicbarker/clay/blob/main/LICENSE.md'}, page_content='1. The origin of this software must not be misrepresented; you must not\n    claim that you wrote the original software. If you use this software in a\n    product, an acknowledgment in the product documentation

In [11]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

db = FAISS.from_documents(chunked_docs, HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5"))

/var/folders/36/2f8494bn15sd97_kz2ghpkrm0000gq/T/ipykernel_24241/965698010.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  db = FAISS.from_documents(chunked_docs, HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5"))
/Users/anirudhsingh/MISC/playground/RAG1/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [ ]:
# directly search the database
query = "how to create a window"
results = db.similarity_search_with_score(query)
for res, score in results:
    print(f"[SIM={score:3f}] {res.page_content} [{res.metadata}]")

[SIM=0.746246] **At startup / initialization time, run once**
`Clay_MinMemorySize` -> `Clay_CreateArenaWithCapacityAndMemory` -> `Clay_SetMeasureTextFunction` -> `Clay_Initialize`

**Each Frame**
`Clay_SetLayoutDimensions` -> `Clay_SetPointerState` -> `Clay_UpdateScrollContainers` -> `Clay_BeginLayout` -> `CLAY() etc...` -> `Clay_EndLayout`

---

### Clay_MinMemorySize

`uint32_t Clay_MinMemorySize()` [{'path': 'README.md', 'sha': 'abecac9da5f394b02a06992263a9d57055af424f', 'source': 'https://api.github.com/nicbarker/clay/blob/main/README.md'}]
[SIM=0.750430] 6. Call [clay.EndLayout(screenWidth, screenHeight)](https://github.com/nicbarker/clay/blob/main/README.md#clay_endlayout) and process the resulting [clay.RenderCommandArray](https://github.com/nicbarker/clay/blob/main/README.md#clay_rendercommandarray) in your choice of renderer.

```Odin
renderCommands: clay.ClayArray(clay.RenderCommand) = clay.EndLayout(windowWidth, windowHeight) [{'path': 'bindings/odin/README.md', 'sha': '579f

In [25]:
# search using the retriever
query = "how to create a window"
docs = retriever.invoke(query)
for doc in docs:
    print(doc.page_content) 

**At startup / initialization time, run once**
`Clay_MinMemorySize` -> `Clay_CreateArenaWithCapacityAndMemory` -> `Clay_SetMeasureTextFunction` -> `Clay_Initialize`

**Each Frame**
`Clay_SetLayoutDimensions` -> `Clay_SetPointerState` -> `Clay_UpdateScrollContainers` -> `Clay_BeginLayout` -> `CLAY() etc...` -> `Clay_EndLayout`

---

### Clay_MinMemorySize

`uint32_t Clay_MinMemorySize()`
6. Call [clay.EndLayout(screenWidth, screenHeight)](https://github.com/nicbarker/clay/blob/main/README.md#clay_endlayout) and process the resulting [clay.RenderCommandArray](https://github.com/nicbarker/clay/blob/main/README.md#clay_rendercommandarray) in your choice of renderer.

```Odin
renderCommands: clay.ClayArray(clay.RenderCommand) = clay.EndLayout(windowWidth, windowHeight)
---

### Clay_BeginLayout

`void Clay_BeginLayout()`

Prepares clay to calculate a new layout. Called each frame / layout **before** any of the [Element Macros](#element-macros).

---

### Clay_EndLayout

`Clay_RenderCommandA

In [13]:
from mlx_lm import load, generate
model, tokenizer = load("mlx-community/Phi-3.5-mini-instruct-4bit")

Fetching 11 files: 100%|██████████| 11/11 [00:00<00:00, 145085.99it/s]


In [ ]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline
from langchain_core.output_parsers import StrOutputParser

prompt_template = """
<|system|>
Answer the question based on your knowledge. Use the following context to help:

{context}

</s>
<|user|>
{question}
</s>
<|assistant|>

 """

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)


In [36]:
query = "How to create a window using clay. Give me a code example"
docs = retriever.invoke(query)
context = ""
for i in range(4):
    context += docs[i].page_content + "\n"
question = query
prompt_str = prompt.format(context=context, question=question)
print(prompt_str)



<|system|>
Answer the question based on your knowledge. Use the following context to help:

<img width="1394" alt="A screenshot of a code IDE with lots of visual and textual elements" src="https://github.com/user-attachments/assets/9986149a-ee0f-449a-a83e-64a392267e3d">

_An example GUI application built with clay_

## Quick Start

Download or clone clay.h and include it after defining `CLAY_IMPLEMENTATION` in one file.

```C
// Must be defined in one file, _before_ #include "clay.h"
#define CLAY_IMPLEMENTATION
#include "../../clay.h"
6. Call [clay.EndLayout(screenWidth, screenHeight)](https://github.com/nicbarker/clay/blob/main/README.md#clay_endlayout) and process the resulting [clay.RenderCommandArray](https://github.com/nicbarker/clay/blob/main/README.md#clay_rendercommandarray) in your choice of renderer.

```Odin
renderCommands: clay.ClayArray(clay.RenderCommand) = clay.EndLayout(windowWidth, windowHeight)
Take a look at the [clay website](https://nicbarker.com/clay) for an exam

In [ ]:
response = generate(model, tokenizer, prompt=prompt_str, verbose=True)

To create a window using `clay`, you first need to define `CLAY_IMPLEMENTATION` and include `clay.h` in your source file. Then, you can use `clay.EndLayout` to specify the size of the window you want to create. Here's a step-byized example of how to create a window with `clay`:

```C
// Define CLAY_IMPLEMENTATION at the beginning of your source file
#define CLAY_IMPLEMENTATION
#include "clay.h"

// ... other code ...

int main() {
    // Set the dimensions for the window (width, height)
    int windowWidth = 800;
    int windowHeight = 600;

    // End the layout with the specified dimensions
    clay.EndLayout(windowWidth, windowHeight);

    // Process the resulting clay.RenderCommandArray in your choice of renderer
    Odin renderCommands: clay.RenderCommandArray() = clay.EndLayout(windowWidth, windowHeight);

    // Your rendering code here

    return
Prompt: 668 tokens, 518.042 tokens-per-sec
Generation: 256 tokens, 82.018 tokens-per-sec
Peak memory: 2.793 GB
